In [60]:
import pandas as pd
import numpy as np
from sklearn import pipeline
from sklearn.linear_model import SGDRegressor, LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.metrics import mean_squared_log_error, mean_squared_error
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor,ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.preprocessing import OneHotEncoder,LabelEncoder, Binarizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from scipy.stats import zscore
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import LocalOutlierFactor
from xgboost import XGBClassifier, XGBRFClassifier

In [2]:
X = pd.read_csv('Training/X_train.csv')
y = pd.read_csv('Training/y_train.csv')

In [3]:
X.shape

(33050, 44)

In [4]:
X['C6'] = X['C6'].apply(lambda x: 0 if x==False else 1)
X['C8'] = X['C8'].apply(lambda x: 0 if x==False else 1)

In [5]:
X = X.iloc[:,1:]
y = y['Dependent_Variable']
X_numerical = X.iloc[:,8:]
numerical_columns= list(X_numerical.columns)
X_categorical = X.iloc[:,:8]
categorical_columns= list(X_categorical.columns)

In [6]:
len(numerical_columns)

35

In [7]:
len(categorical_columns)

8

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

In [9]:
all_feature = numerical_columns.copy()
all_feature.extend(categorical_columns)

In [10]:
len(all_feature)

43

### Using all featues

In [11]:
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
          ('poly', PolynomialFeatures(degree=1)),
        ('std', StandardScaler()),
    ])

cat_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="most_frequent")),
          ('onehot', OneHotEncoder(handle_unknown='ignore')),
    ])

column_combine = ColumnTransformer([
        ("num", num_pipeline, numerical_columns),
        ("cat", cat_pipeline, categorical_columns),
    ])

full_pipeline = Pipeline(steps=[('data_pre_process', column_combine), ('model', LogisticRegression())])
history = full_pipeline.fit(X_train,y_train)

c:\users\user\anaconda3\envs\tensorflow_2_6\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [12]:
y_train_proba = full_pipeline.predict_proba(X_train)
y_test_proba = full_pipeline.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_train, y_train_proba[:,1], pos_label=1)
train_auc =  auc(fpr, tpr)
fpr, tpr, thresholds = roc_curve(y_test, y_test_proba[:,1], pos_label=1)
test_auc =  auc(fpr, tpr)
print("Train AUC is: ",train_auc)
print("Test AUC is: ",test_auc)

Train AUC is:  0.7705835206700378
Test AUC is:  0.7587575880165814


### Try selected features from EDA

In [13]:
numerical_columns_from_EDA = list(set(['N1',
 'N2',
 'N3',
 'N4',
 'N5',
 'N6',
 'N7',
 'N8',
 'N9',
 'N10',
 'N10.1',
 'N11',
 'N12',
 'N14',
 'N15',
 'N16',
 'N17',
 'N18',
 'N19',
 'N20',
 'N21',
 'N22',
 'N23',
 'N24',
 'N25',
 'N26',
 'N27',
 'N28',
 'N29',
 'N30',
 'N31',
 'N32',
 'N33',
 'N34',
 'N35']) - set(['N26','N27','N28','N29','N6','N3']))

In [14]:
categorical_columns_from_EDA = list(set(['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'])-set(['C3','C6','C8']))

In [15]:
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
          ('poly', PolynomialFeatures(degree=1)),
        ('std', StandardScaler()),
    ])

cat_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="most_frequent")),
          ('onehot', OneHotEncoder(handle_unknown='ignore')),
    ])

column_combine = ColumnTransformer([
        ("num", num_pipeline, numerical_columns_from_EDA),
        ("cat", cat_pipeline, categorical_columns_from_EDA),
    ])

full_pipeline = Pipeline(steps=[('data_pre_process', column_combine), ('model', LogisticRegression())])
history = full_pipeline.fit(X_train,y_train)

c:\users\user\anaconda3\envs\tensorflow_2_6\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [16]:
y_train_proba = full_pipeline.predict_proba(X_train)
y_test_proba = full_pipeline.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_train, y_train_proba[:,1], pos_label=1)
train_auc =  auc(fpr, tpr)
fpr, tpr, thresholds = roc_curve(y_test, y_test_proba[:,1], pos_label=1)
test_auc =  auc(fpr, tpr)
print("Train AUC is: ",train_auc)
print("Test AUC is: ",test_auc)

Train AUC is:  0.760554242730847
Test AUC is:  0.7526109677027044


### Try outlier removal from EDA

In [17]:
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std', StandardScaler()),
    ])

cat_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="most_frequent")),
    ])

column_combine = ColumnTransformer([
        ("num", num_pipeline, numerical_columns),
        ("cat", cat_pipeline, categorical_columns),
    ])

full_pipeline = Pipeline(steps=[('data_pre_process', column_combine)])
TEMP = full_pipeline.fit(X_train,y_train)

In [18]:
TEMP = full_pipeline.fit_transform(X_train,y_train)

In [19]:
data = pd.DataFrame(TEMP,columns=all_feature)
data.shape

(22143, 43)

In [20]:
from sklearn.neighbors import LocalOutlierFactor
clf = LocalOutlierFactor(n_neighbors=20)
clf.fit_predict(data)
negative_outlier_factor = clf.negative_outlier_factor_

In [21]:
negative_outlier_factor.max()

-0.9544700853825031

In [22]:
negative_outlier_factor.min()

-8.116913085035561

In [23]:
data_outlier_removed = pd.concat([data.reset_index(),y_train.reset_index()],axis=1)
data_outlier_removed = data_outlier_removed.drop(columns=['index'])
data_outlier_removed.shape

(22143, 44)

In [24]:
data_outlier_removed['is_outlier'] = negative_outlier_factor
data_outlier_removed = data_outlier_removed[data_outlier_removed['is_outlier']>-7]
print("Loss of data in train is: ", (1-(data_outlier_removed.shape[0]/data.shape[0]))*100)

Loss of data in train is:  0.004516099896134662


In [25]:
data_outlier_removed = data_outlier_removed.drop(columns=['is_outlier'])

In [26]:
y_train_out = data_outlier_removed['Dependent_Variable']
X_train_out = data_outlier_removed.drop(columns=['Dependent_Variable'])

In [27]:
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
          ('poly', PolynomialFeatures(degree=1)),
        ('std', StandardScaler()),
    ])

cat_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="most_frequent")),
          ('onehot', OneHotEncoder(handle_unknown='ignore')),
    ])

column_combine = ColumnTransformer([
        ("num", num_pipeline, numerical_columns),
        ("cat", cat_pipeline, categorical_columns),
    ])

full_pipeline = Pipeline(steps=[('data_pre_process', column_combine), ('model', LogisticRegression())])
history = full_pipeline.fit(X_train_out,y_train_out)

c:\users\user\anaconda3\envs\tensorflow_2_6\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [28]:
X_train_out.head()

,N1,N2,N3,N4,N5,N6,N7,N8,N9,N10,...,N34,N35,C1,C2,C3,C4,C5,C6,C7,C8
0,-1.225387,0.304623,1.792958,1.070056,1.235184,1.792958,1.824841,1.747241,-0.185805,-0.340505,...,-0.162440,0.468606,1.0,15.0,26.0,1.0,1.0,1.0,2.0,1.0
1,-1.434033,-0.376256,1.792958,-0.088833,-0.060146,1.792958,0.344433,-0.014250,-0.482090,-0.340505,...,-0.440927,-0.351626,1.0,1.0,2.0,41.0,2.0,1.0,2.0,1.0
2,0.443781,-0.514953,-0.622620,-0.088833,-0.060146,-0.622620,-0.572010,-0.014250,-0.404702,-0.340505,...,-0.573440,-0.351626,1.0,0.0,14.0,1.0,2.0,0.0,4.0,1.0
3,0.126491,-0.666260,-1.347293,1.649500,1.882849,-1.347293,1.190381,1.527054,0.457621,-0.340505,...,-0.292057,0.878722,1.0,0.0,15.0,6.0,2.0,1.0,2.0,1.0
4,-0.711797,0.783759,-0.864178,-0.475129,-0.276034,-0.864178,-1.065479,-0.014250,0.095003,-0.340505,...,-0.924966,-0.488331,1.0,7.0,48.0,17.0,2.0,0.0,4.0,1.0


In [29]:
X_test_out = X_test.copy()
X_test_out.columns = X_train_out.columns

In [30]:
y_train_proba = full_pipeline.predict_proba(X_train_out)
y_test_proba = full_pipeline.predict_proba(X_test_out)
fpr, tpr, thresholds = roc_curve(y_train_out, y_train_proba[:,1], pos_label=1)
train_auc =  auc(fpr, tpr)
fpr, tpr, thresholds = roc_curve(y_test, y_test_proba[:,1], pos_label=1)
test_auc =  auc(fpr, tpr)
print("Train AUC is: ",train_auc)
print("Test AUC is: ",test_auc)

Train AUC is:  0.7705523807871115
Test AUC is:  0.530392414708756


**`Removing outlier performs good on train but test AUC is less. I am planning to keep all the points.`**

### Try different models

`As EDA did not help much lets try different models on all data. I will do that in next notebook with hyper parameter tuning`

#### 1. Randomforest

In [31]:
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
          ('poly', PolynomialFeatures(degree=1)),
        ('std', StandardScaler()),
    ])

cat_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="most_frequent")),
          ('onehot', OneHotEncoder(handle_unknown='ignore')),
    ])

column_combine = ColumnTransformer([
        ("num", num_pipeline, numerical_columns),
        ("cat", cat_pipeline, categorical_columns),
    ])

full_pipeline = Pipeline(steps=[('data_pre_process', column_combine), ('model', RandomForestClassifier())])
history = full_pipeline.fit(X_train,y_train)

In [32]:
y_train_proba = full_pipeline.predict_proba(X_train)
y_test_proba = full_pipeline.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_train, y_train_proba[:,1], pos_label=1)
train_auc =  auc(fpr, tpr)
fpr, tpr, thresholds = roc_curve(y_test, y_test_proba[:,1], pos_label=1)
test_auc =  auc(fpr, tpr)
print("Train AUC is: ",train_auc)
print("Test AUC is: ",test_auc)

Train AUC is:  1.0
Test AUC is:  0.7667201021828219


### Try PCA

#### With only numerical features

In [33]:
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
          ('poly', PolynomialFeatures(degree=1)),
        ('std', StandardScaler()),
    ])

cat_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="most_frequent")),
          ('onehot', OneHotEncoder(handle_unknown='ignore')),
    ])

column_combine = ColumnTransformer([
        ("num", num_pipeline, numerical_columns),
    ])

full_pipeline = Pipeline(steps=[('data_pre_process', column_combine)])

In [34]:
X_train_PCA = full_pipeline.fit_transform(X_train,y_train)

In [35]:
X_test_PCA = full_pipeline.transform(X_test)

In [36]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.99)
X_train_pca = pca.fit_transform(X_train_PCA)
X_test_pca = pca.transform(X_test_PCA)

In [37]:
X_train.shape

(22143, 43)

In [38]:
X_train_pca.shape

(22143, 30)

In [39]:
model_lr = LogisticRegression()
model_lr.fit(X_train_pca,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [40]:
y_train_proba = model_lr.predict_proba(X_train_pca)
y_test_proba = model_lr.predict_proba(X_test_pca)
fpr, tpr, thresholds = roc_curve(y_train, y_train_proba[:,1], pos_label=1)
train_auc =  auc(fpr, tpr)
fpr, tpr, thresholds = roc_curve(y_test, y_test_proba[:,1], pos_label=1)
test_auc =  auc(fpr, tpr)
print("Train AUC is: ",train_auc)
print("Test AUC is: ",test_auc)

Train AUC is:  0.7426755704114802
Test AUC is:  0.7410426983171278


#### With numerical and categorical features

In [48]:
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
          ('poly', PolynomialFeatures(degree=1)),
        ('std', StandardScaler()),
    ])

cat_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="most_frequent")),
          ('onehot', OneHotEncoder(handle_unknown='ignore')),
    ])

column_combine = ColumnTransformer([
        ("num", num_pipeline, numerical_columns),
        ("cat", cat_pipeline, categorical_columns),
    ])

full_pipeline = Pipeline(steps=[('data_pre_process', column_combine)])

In [49]:
X_train_PCA = full_pipeline.fit_transform(X_train,y_train)

In [50]:
X_test_PCA = full_pipeline.transform(X_test)

In [52]:
X_train_PCA

<22143x200 sparse matrix of type '<class 'numpy.float64'>'
	with 952149 stored elements in Compressed Sparse Row format>

In [54]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.99)
X_train_pca = pca.fit_transform(X_train_PCA.toarray())
X_test_pca = pca.transform(X_test_PCA.toarray())

In [55]:
X_train_PCA.shape

(22143, 200)

In [56]:
X_train_pca.shape

(22143, 94)

In [57]:
model_lr = LogisticRegression()
model_lr.fit(X_train_pca,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [58]:
y_train_proba = model_lr.predict_proba(X_train_pca)
y_test_proba = model_lr.predict_proba(X_test_pca)
fpr, tpr, thresholds = roc_curve(y_train, y_train_proba[:,1], pos_label=1)
train_auc =  auc(fpr, tpr)
fpr, tpr, thresholds = roc_curve(y_test, y_test_proba[:,1], pos_label=1)
test_auc =  auc(fpr, tpr)
print("Train AUC is: ",train_auc)
print("Test AUC is: ",test_auc)

Train AUC is:  0.7641632616088638
Test AUC is:  0.7551516752632945


### Try XGBoost

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

In [71]:
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
          ('poly', PolynomialFeatures(degree=1)),
        ('std', StandardScaler()),
    ])

cat_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="most_frequent")),
          ('onehot', OneHotEncoder(handle_unknown='ignore')),
    ])

column_combine = ColumnTransformer([
        ("num", num_pipeline, numerical_columns),
        ("cat", cat_pipeline, categorical_columns),
    ])

full_pipeline = Pipeline(steps=[('data_pre_process', column_combine), ('model', XGBClassifier())])
history = full_pipeline.fit(X_train,y_train)

c:\users\user\anaconda3\envs\tensorflow_2_6\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:58:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [72]:
y_train_proba = full_pipeline.predict_proba(X_train)
y_test_proba = full_pipeline.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_train, y_train_proba[:,1], pos_label=1)
train_auc =  auc(fpr, tpr)
fpr, tpr, thresholds = roc_curve(y_test, y_test_proba[:,1], pos_label=1)
test_auc =  auc(fpr, tpr)
print("Train AUC is: ",train_auc)
print("Test AUC is: ",test_auc)

Train AUC is:  0.9413591619902961
Test AUC is:  0.7584976766892917
